# **Analysis of Film Industry ROIs**
## Malcolm Katzenbach, Lauren Phipps, Dan Valenzuela

***



## **Overview** <a id="Overview"></a>

This project analyzes the budgets and returns on investment for movies ranging from 2010-2019. Using data from the [Internet Movie Database](https://www.imdb.com/) and [The-Numbers.com](https://www.the-numbers.com/), the movies were broken down into genre and budget categories to determine any trends in returns on investment. This information will be used to help Microsoft's new movie studio maximize their success by selecting genres and budget tiers that will yield the highest return on their investment.

[**1. Business Problem**](#Business-Problem)

[**2. Data Understanding**](#Data-Understanding)

[**3. Data Preparation**](#Data-Preparation)

[**4. Data Analysis**](#Data-Analysis)

[**5. Evaluation**](#Evaluation)

[**6. Conclusion**](#Conclusion)
***

## **Business Problem** <a id="Business-Understanding"></a>
[*↑ Back to overview*](#Overview)

Many large companies have started to venture into creating original video content. Seeing the potential and wanting to be able to compete with other companies, Microsoft has created a new movie studio. In an effort to be as successful as possible, research must be done into the movie industry to determine what conditions will yield the highest return on investment (ROI). Microsoft's goal is to maximize their return on investment for the movies they create. 

The business questions that will be addressed are:

  1) What is the relationship between a film's budget and it's return on investment?

  2) What is the distribution of returns on investment for budget tiers for films?

  3) What is the distribution of returns on investment for movie genres?

  4) Which budget tier for each genre will yield the highest median return on investment?

[*↑ Back to overview*](#Overview)
***

## **Data Understanding** <a id="Data-Understanding"></a>
[*↑ Back to overview*](#Overview)

### Datasets
For the purposes of this analysis we focused primarily on data from the Internet Movie Database (IMDB) and The-Numbers.com (TN), two sources that focus on the film industry. Specifically we used datasets that included--on one hand--title, date released, and genre data and--on the other hand--title, date released, production budgets, and box office figures. Below is a summary of the data pertinent to our analysis broken down by file. 

| imdb.title.basics.csv | tn.movie_budgets.csv |
| --- | --- |
| primary_title | movie |
| start_year | release_date |
| genres |  |
|  | production_budget |
|  | domestic_gross |
|  | worldwide_gross |


In this table you can see our understanding of how data between the two datasets can "match" in the sense that they provide they same kind of data but may be in different formats. For example, `start_year` in `imdb.title.basics.csv` is  formatted as `YYYY` whereas `release_date` in `tn.movie_budgets.csv` is formatted as `MMM DD, YYYY`. You can see below examples of such a discrepancy.

In [3]:
import CustomLibrary as cl
from CustomLibrary import df_title_basics, df_movie_budgets

df_title_basics.head(3)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama


In [ ]:
df_movie_budgets.head(3)

Further, there are issues with being able to utilize the data due to their data types. The previously discussed `release_date` and `start_year` data are actually `objects` and `integers`, respectively. And much of the box office data are `objects` that can't be added and subtraced. How we dealt with them can be seen in the [data preparation](#Data-Preparation) section.

In [ ]:
cl.df_info()

### Key Data for Merging Datasets and Analysis

Another key issue for this data is identifying which data we can use to perform a merge. Although the `imdb.title.basics.csv` contains a unique identifier column `tconst` the `tn.movie_budgets.csv` dataset contains no unique IDs. We worked around that issue by identifying the title variables `primary_title` and `movie` and release date variables `start_year`and `release_date` variables as our keys. As mentioned before these variables did not match in terms of format so, for example, we identified the year string in `release_date` as data that will be used to match release date variables. 

Although the datasets contain key variables as `genres`, `production_budget`, and `worldwide_gross`, the datasets also lack total costs of producing each movie, including costs like marketing, and more granular genre data. As seen in [data preparation](#Data-Preparation), we made key assumptions, calculations, and manipulations to make the dataframe accessible for our analysis of ROI, genre, and budgets. 

[*↑ Back to overview*](#Overview)
***

## **Data Preparation** <a id="Data-Preparation"></a>
[*↑ Back to overview*](#Overview)

[*↑ Back to overview*](#Overview)
***

## **Data Analysis** <a id="Data-Analysis"></a>
[*↑ Back to overview*](#Overview)

### Question 1<a id="Question-1"></a> - What is the relationship between a film's budget and it's return on investment?

The first step in the analysis was to determine if there is a correlation between the amount of money invested in a movie and the return on investment. This will help decide if it is more beneficial to invest in larger blockbuster movies or if there is more return for smaller, low budget films. A scatterplot was created with the film's budget (production and advertisement costs) on the x-axis and the return on investment as a percentage on the y-axis. This was then further broken down by the low, mid, and high budget classification to more clearly show the divide between the budgets.

In [ ]:
budget_ROI_scatter(DataFrame)

This plot shows that there is no strong correlation between a film's budget and it's return on investment. The correlation coefficient is -0.05, again confirming no correlation. Generally, more money invested in a film does not automatically lead to a higher return on that investment. This was then broken down into three categories: low, mid, and high budgets.

In [ ]:
LB_ROI_scatter(DataFrame)

As the plot above shows, there is no correlation between movie budget and return on investment for low budget films (movies with budgets under $ 25 million). The correlation coefficient was approximately -0.1. 

In [ ]:
MB_ROI_scatter(DataFrame)

As the plot above shows, there is no correlation between movie budget and return on investment for mid-range budget films (movies with budgets between 25 million and 100 million dollars). The correlation coefficient was approximately -0.1.

In [ ]:
HB_ROI_scatter(DataFrame)

As the plot above shows, there is no correlation between movie budget and return on investment for low budget films (movies with budgets over $100 million). The correlation coefficient was approximately 0.05. 

A movie's budget alone does not affect it's return on investment. 

### Question 2 - What is the distribution of ROI by budget tiers? <a id="Question-2"></a>

The next step was to look more closely at the distribution of ROI's for the films in each budget tier to determine the median value and look for trends. Due to outliers, the median value was used as a measure of the general tendancy of the data. A boxplot was used to look at the characteristics of each budget tier. Outliers are not shown in this plot, but are still part of the data. Their impact can be seen from the range of the whiskers. The median is represented by the green line. 

In [ ]:
ROI_budget_boxplot(DataFrame)

This graph shows that the median is highest for high budget films and that only a small percentage of those movies have a negative ROI and would lose money. They are the safest investment to get a postive ROI. Low budget films do have the highest portion of movies that will lose money, but at the same time, have the highest range of positive ROI's. The upper end of the box denotes the 75th percentile and that is at 200%, while the whiskers denoting the 90th percentile is significantly higher at over 600% return. This shows that there is potential to get a very high return on an investment in a low budget film. 

### Question 3<a id="Question-3"></a> - What is the distribution of the returns on investment for each movie genre?

When investing in movies, it is important to know if certain genres lead to a higher return on investment than others. To answer this question, the data was broken into 7 genres. These were the most common genres contained in the data set. Some movies were categorized as more than one genre. Those movies are included in each of the listed genres. For example, if a movie is considered Action and Adventure, it's data in included in both genres because it is representative of both genres. The final category is "other" for any movies that did not fall into one of the seven listed genres or did not specify a genre. Only 25 movies were included in this category.

Because there is a wide range of values for returns on investment, it is beneficial to use a boxplot to get a more complete look at the data for each genre. Outliers are not shown on this plot, but are still part of the data. Their impact can be seen from the range of the whiskers. The median is represented by the red line. Because of the significant outliers, the median is a better representation of the data than the mean.

In [ ]:
create_genre_dfs(DataFrame)
genre_boxplot('ROI')

With the exception of documentaries, all genres have a positive median return on investment, meaning more than half of the movies in each of those genres will make a profit. Adventure and thriller movies have the highest median ROI. Overall, adventure films are the most likely to have a positive ROI, which is shown by having the highest amount of their box and top whisker above 0%. However, thriller movies have the highest range of positive ROI's, with it's 90th percentile (shown as the top whisker) reaching over 400%. This shows that thriller movies do have the potential to be highest return on an investment. 

### Question 4 - Which budget tier for each genre will yield the highest median return on investment?<a id="Question-4"></a> 

As a final step, it was beneficial to use the budget tiers from Questions 1 and 2 to further analyze the specific genres of films to determine if certain genres with specific budgets created more successful films.

In [ ]:
LB_Genres_Graph(DataFrame)

This graph shows that, for low budget films, most genres do not have a positive median ROI. The exception is for low-budget thriller films. These yield a median ROI of 100%, which corresponds with doubling the initial investment in the film. 

In [ ]:
MB_Genres_Graph(DataFrame)

Adventure, comedy, and thriller movies with a mid-range budget have a modestly positive median ROI, with values ranging from 10-20% ROI. Documentaries with a mid-range budget yielded the lowest median ROI at -30%, showing that most of lost money. 

In [ ]:
HB_Genres_Graph(DataFrame)

High budget films across all genres yielded a positive ROI, with adventure, comedy, and thriller films yielding the highest. This graph confirms that most films that have a budget of over $100 million will have a positve return on that investment. 

[*↑ Back to overview*](#Overview)
***

## **Evaluation**<a id="Evaluation"></a>
[*↑ Back to overview*](#Overview)

The overall business problem that was given asked how a new movie studio may be as successful as possible in terms of ROI. By looking more deeply into the data on movies from the past 10 years, the analysis has shown what types of genres have done well with certain sized budgets. This ha successfully answered the main question in a basic way. This analysis is helpful as done, however there are certain areas that could bring more accuracy to the analysis. 

- **The Advertisement Cost**: In the analysis, it was assumed that advertisement budget was equal to the production budget. However, it unlikely that all movies had an equal advertisement and production budget. By using more accurates advertisement budgets, it will increase the accuracy of the ROI.
- **Time Period of Analysis**: The dataframes were only for movies from the last 10 years.     While useful for immediate future movies, having a longer period of data could increase how robust our analysis is.
- **Inflation**: This analysis did not account for inflation. While only over a 10 year period, the inflation would have an effect on the money values of older movies, which would increase the accuracy of the comparison between  movies.
- **Box Office Only**: This data only takes into account box office revenues. If Microsoft desired to go into streaming services and produce movies for the streaming service only, it would be more accurate to include data on streamed movies.

[*↑ Back to overview*](#Overview)
***

## **Conclusion**<a id="Conclusion"></a>
[*↑ Back to overview*](#Overview)

This analysis of ROI across budget tiers and genres brings forward two recommendations:

**Produce high budget adventure films-** if the studio has a high production & advertisement budget, it would make sense for the studio to invest in an adventure film since it had the highest median ROI for high budgets.

**Produce low budget thrillers-** if the studio has a low budget, it would make sense for the studio in thrillers since they have the highest possible ROI of any genre film with low budgets. 

[*↑ Back to overview*](#Overview)
***

## **Next Steps**
[*↑ Back to overview*](#Overview)

Based on our evaluation and conclusion, there are multiple paths for further analysis. We suggest the following three possible avenues:

**Evaluating personnel that maximize ROI-** By analyzing the personnel of successful movies in genres such as thrillers and adventures, the studio could attempt to hire those that would most likely help produce a movie with a high ROI.

**Find accurate advertisement budgets for movies and account for inflation-** By going back into the analysis and improving cost data, the accuracy of the ROIs could be increased.

**Constructing Portfolios of Films-** With a given studio budget, a portfolio of films could be maximized for ROI by genre and budget for each.

[*↑ Back to overview*](#Overview)
***